# MAP - Multi Agent Planning
##### Aviv Cohen & Dan Navon

## Introduction
Multiagent planning is concerned with planning by (and for) multiple agents. It can involve agents planning for a common goal, an agent coordinating the plans (plan merging) or planning of others, or agents refining their own plans while negotiating over tasks or resources.

## Motivation
On 236609 - MULTI ROBOT SYSTEMS - Sarah's previous course, we faced the collaborative inspection problem where 2 agents needs to collaborate in order to count as many spheres as possible.

<img src='img/inspection.jpg' width=500/>

Our single agent solution implementation dependent on splitting the graph such that both agent will finish revealing the scene on the shortest possible time.
Since we faced difficulties we implemented it naively were each robot goes to its closest area.

<img src='img/Motivation.jpeg' width=500/>



# The problem

State space - $\mathcal{S} = S^0 \times ... \times S^k \times ... \times S^{k+i} \times ... \times S^{k+i+j} $ where $k$ is the number of taxis, $i$ the number of passengers and $j$ number of their destination. $S$ represents the map's dimensions.
joint action space - $\mathcal{A} = A^0 \times ... \times A^k $ taxis and $A$ represent the number of actions each taxi can perform.

The joint graph will describe each possible expansion of $s \in \mathcal{S}$ using $a \in \mathcal{A}$ to $s' \in \mathcal{S}$

### Current settings
Taxis will work in a cooperative / collaborative settings with centralized control, fully observable and implicit communication
In order to test our algorithms, whe chose a domain 4x4 map with 2 taxis and 2 passengers.
$|\mathcal{S}| = (4\cdot 4)^{2+2+2} = 16^6 = 16,777,216$
$|\mathcal{A}| = 7^2 = 49$

We can now understand why MAP is such a complex task, even for a relatively small size of domain our space-state action is huge.

### Pruning

In order to reduce the amount of nodes we'll explore, we tested whether the state will be changed after each joint action.
 If picking or dropoff won't change taxi's location we omit this action before its processed into successors data structure.
 The chosen domain is small and contain a few barriers at the middle, we see that we usually have around 12 different joint actions which greatly reduce the running time.

### Makespan

Makespan of a project is the length of time that elapses from the start of work to the end.
In our scenario, makespan related to the longest path to be executed by one of the agents.

## Multi Agent BFS
In order to find a solution to the makespan problem, one can use BFS which guarantee to find a solution if it exists.
The advantage of using BFS is its general purpose which can use to achieve different goals such as solution to the makespan issue or collaboration, but it comes with a price of expanding multiple nodes.

We would like to start with performing uninformed search as a benchmark over the state-joint action graph

In [1]:
# not needed with provided conda environment
# !pip install git+https://github.com/sarah-keren/multi_taxi
# !pip install git+https://github.com/sarah-keren/AI_agents

## Environment
We'll test our MultiAgentPlanning algorithms over the taxi environment we've already seen with the use of best_first_search as the search backbone for A* and BFS.


In [2]:
import time
import numpy as np

from copy import deepcopy
from IPython.display import clear_output

from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_bipartite_matching

from multi_taxi import MultiTaxiEnv
from AI_agents.Search.best_first_search import breadth_first_search, a_star
from MAP import MapProblem
from copy import deepcopy
import copy


MAP2 = [
    "+-------+",
    "| : |F: |",
    "| : | : |",
    "| : : : |",
    "| | :G| |",
    "+-------+",
]

taxi_env = MultiTaxiEnv(num_taxis=2, num_passengers=2, domain_map=MAP2)

In order to be compatible with both AI_agents library and the TaxiEnvironment we have to switch the state represntation between list of lists and tuple of tuples

In [3]:
def list_to_tuple(x):
    lolol = deepcopy(x) # list of lists of lists
    result = list()
    while lolol:
        lol = lolol.pop(0)
        # single list
        if type(lol[0]) is not list:
            result.append(tuple(lol))

        # list of lists
        else:
            local_res = list()
            while lol:
                l = lol.pop(0)
                local_res.append(tuple(l))
            result.append(tuple(local_res))

    return tuple(result)


def tuple_to_list(x):
    totot = list(x)
    result = list()
    while totot:
        tot = totot.pop(0)
        if type(tot[0]) is not tuple:
            result.append(list(tot))
        else:
            tot = list(tot)
            local_res = list()
            while tot:
                t = tot.pop(0)
                local_res.append(list(t))
            result.append(list(local_res))

    return result

In [4]:
def run_plan(state, plan, problem):
    new_state = deepcopy(state)
    problem.env.reset()
    problem.set_state(state)
    problem.env.render()
    for action in plan:
        time.sleep(0.25)
        new_state = problem.step(eval(action))
        clear_output(wait=True)
        problem.env.render()

    return new_state

Initializing MapProblem class to describe our Multi Agent Planning problem.

In [5]:
taxi_env.reset()
initial_state=[[[2, 2], [2, 1]], [np.inf, np.inf], [[3, 1], [3, 3]], [[1, 3], [1, 2]], [2, 2]]
# initial_state=deepcopy(taxi_env.state)
map_problem = MapProblem(taxi_env, list_to_tuple(initial_state))
map_problem.set_state(initial_state)
taxi_env.render()

+-------+
| : |F: |
| : |D:D|
| :_:_: |
| |P:G|P|
+-------+
Taxi0-YELLOW: Fuel: inf, Location: (2,2), Collided: False
Taxi1-RED: Fuel: inf, Location: (2,1), Collided: False
Passenger1: Location: (3, 1), Destination: (1, 3)
Passenger2: Location: (3, 3), Destination: (1, 2)
Done: False, {'taxi_0': False, 'taxi_1': False, '__all__': False}
Passengers Status's: [2, 2]


## Executing BFS

In [6]:
plan = breadth_first_search(map_problem)
[best_value, best_node, path, explored_count, ex_terminated] = plan
print('path length:', len(path))
print('explored_count:', explored_count)
explored_count

path length: 8
explored_count: 600


600

In [7]:

plan = breadth_first_search(map_problem)
[best_value, best_node, path, explored_count, ex_terminated] = plan
print('path length:', len(path))
print('explored count:', explored_count)

path length: 8
explored count: 600


In [8]:
s = deepcopy(initial_state)
final_state = run_plan(s,path, map_problem)
print('path length:', len(path))

+-------+
| : |F: |
| : |_:_|
| : : : |
| | :G| |
+-------+
Taxi0-YELLOW: Fuel: inf, Location: (1,3), Collided: False
Taxi1-RED: Fuel: inf, Location: (1,2), Collided: False
Passenger1: Location: Arrived!, Destination: (1, 3)
Passenger2: Location: Arrived!, Destination: (1, 2)
Done: True, {'taxi_0': True, 'taxi_1': True, '__all__': True}
Passengers Status's: [1, 1]
path length: 8


### Too slow! can we do better?
Yes! we've already seen it with at the single agent planning tutorial.
This time, as in BFS, we expand each node according to its current distance from the base node with additional heuristic function which estimates the distance of the agents to their passengers and their destination.



In [9]:
def joint_simulation(state, h, print_simulation=False, domain_m=MAP2):
    taxi_P = MultiTaxiEnv(num_taxis=len(state[0]), num_passengers=len(state[2]), domain_map=domain_m)
    map_p = MapProblem(taxi_P, list_to_tuple(state))

    [_, _, path, explored_count, _] = a_star(problem=map_p, heuristic_func=h)
    if print_simulation:
        run_plan(state,path,map_p)
        print('explored_count:', explored_count)
        print('path length:', len(path))
    return path

### But what about the heuristic?
Designing a good heuristic for a multi agents system isn't as trivial as one might think.
We'll now present a simple approach for admissible heuristic - the shortest distance from one of thetaxis to
we can try something like ...

In [10]:
class MultiAgentsHeuristic:
    def __init__(self, single_agent_heuristic, aggr_func ):
        self.h = single_agent_heuristic
        self.aggr_func = aggr_func

    def __call__(self, node):
        """
        return an object that presents the joint heuristic of this state, by using the heuristic of one agent and one task
        """
        state = node.state.key
        taxis_src = state[0]
        passengers_src = state[2]
        passengers_dst = state[3]
        passengers_status = state[4]
        values_mat = np.array([[self.h(taxi_id, taxi_src, passenger_src, passenger_dst, passenger_status)
                                for passenger_src, passenger_dst, passenger_status
                                in zip(passengers_src, passengers_dst, passengers_status)]
                               for taxi_id, taxi_src in enumerate(taxis_src)])

        # values, match = allocate_tasks(values_mat)
        g_score = self.aggr_func(values_mat) + node.path_cost
        return g_score

def manhattan_distance(p, q):
    return abs(p[0] - q[0]) + abs(p[1] - q[1])


def manhattan_heuristic(taxi_id, taxi_src, passenger_src, passenger_dst, passenger_status):
    """
    manhatten distance to from the taxi's source to the passenger's source, and from there to the passenger's destination
    """
    is_waiting = passenger_status == 2
    not_waiting = passenger_status != 2
    has_arrived = passenger_status == 1
    not_arrived = passenger_status != 1
    in_taxi = taxi_id + 3 == passenger_status
    return (manhattan_distance(taxi_src, passenger_src) + manhattan_distance(passenger_src, passenger_dst)) * is_waiting \
           + manhattan_distance(taxi_src, passenger_dst) * in_taxi + (2 - has_arrived - not_waiting)


In [11]:
def closest_passenger(values_mat):
    return np.min(values_mat)

In [12]:
# A* code with a simple reliable heuristic like the distance between the closest (taxi, passenger) pair

s = deepcopy(initial_state)
mah = MultiAgentsHeuristic(single_agent_heuristic=manhattan_heuristic,aggr_func=closest_passenger)
path = joint_simulation(s,mah,print_simulation=True)


+-------+
| : |F:_|
| : |_: |
| : : : |
| | :G| |
+-------+
Taxi0-YELLOW: Fuel: inf, Location: (0,3), Collided: False
Taxi1-RED: Fuel: inf, Location: (1,2), Collided: False
Passenger1: Location: Arrived!, Destination: (1, 3)
Passenger2: Location: Arrived!, Destination: (1, 2)
Done: True, {'taxi_0': True, 'taxi_1': True, '__all__': True}
Passengers Status's: [1, 1]
explored_count: 1086
path length: 12


## Better than before, but still slow. can we do better?


## The Decentralized Approach

another approach to solve the planning problem is to use a decentralized method, and distribute the computations.
for using that method, we have to reduce the generalization and to restrict our environment.

our environment will contain n taxis and k passengers, such that n >= k, and each taxi could take one passenger at most.
additionally, we will assume there is no interaction between the taxis.

in this setting, our decentralized algorithm is:

1. separate each pair of taxi and passenger to an isolated environment
2. compute the cost of each pair individually (the number of actions the taxi has to do to take the passenger to his destination).
3. allocate the tasks to the agents, in an optimal way.

This approach can be distributed naturally, by separating the planning for each pair.

what do we gain by using this approach except the distributed computations?

## Comparison Between Centralized And Decentralized Approaches Complexity

let's say that the size of the optimal path is p, and the branching factor of each taxi is b.
if we use the BFS algorithm, the complexity of the centralized approach is $O((b^2)^p) = O(b^{2p})$, while in the decentralized approach, the complexity of each taxi is $O(b^pnk)$.

## Task Allocation

Now we will focus the third part of the decentralized method.
Our task is to find an optimal matching between the taxis and the passengers.

We can present out input as a bipartite fully connected weighted graph G = (U,V,W), where each node u in U represents a taxi, each node v in V represents a passenger, and each weight w(u,v), represents the cost of taking the passenger v to his destination, by using the taxi u.

<img height="600" src="img/bipartite graph.png" width="300"/>

let's formalize our goal.
to do this, we will use the [matching](https://en.wikipedia.org/wiki/Matching_(graph_theory)) definition of the graph theory.
one of our constraints is that we have to allocate a taxi for each passenger. that means, we have to find a maximal matching.
given $M,M'$ maximal matches, let's define $M \geq M'$, if the maximal edge of $M$ is greater than or equal to the maximal edge of $M'$.
our goal is to optimize the makespan problem, that is we want to find a maximal matching $M$, such that for every maximal matching $M'$, $M \leq M'$. from now on, we will call this matching the best makespan matching.

## A trivial algorithm to find the best makespan matching
we can do an exhaustive search over all the possible maximal matches to find the best one.
the number of the possible maximal matches is ${n \choose k}k!$

## An iterative algorithm to find the best makespan matching

let's suppose we know how to find a maximal matching in an unweighted bipartite graph (we will see how to that later).

iterative algorithm:
    1. ignore the weights of G, and find a maximal matching M.
    2. remove the maximal edge from G.
    3. ignore the weights of G, and find a maximal matching M'.
    4. if size(M) = size(M'), then M <- M', and jump to 2. else, return M.

# Correctness of the algorithm
if there is a better matching, it cannot contain the highest value edge, so we can remove it.

## An optimization of the algorithm
instead of removing one edge each iteration, we can do a binary search- for each iteration, if the maximal matching is equals to the number of the passengers, then remove half of the edges with the highest value. otherwise, add half of the lowest value edges we removed at the last iteration.

## How can we find a maximal matching in a bipartite graph?
reminder from algorithms course: there is a reduction from the maximal matching problem to the maximal flow problem.

<img height="600" src="img/flow network.png" width="800"/>

we can solve this [maximal flow problem](https://en.wikipedia.org/wiki/Maximum_flow_problem) with the [Ford-Fulkerson algorithm](https://en.wikipedia.org/wiki/Ford%E2%80%93Fulkerson_algorithm) for example.

## The optimized iterative algorithm's complexity
we have at most $log(|E|)$ iterations, and each iteration takes $O(|E||v|)$ (Ford-Fulkerson complexity). that means $O(|E||V|log(|V|))$.
since the $|E| = nk$, and $|V| = n + k = O(n)$, the complexity is $O(kn^2log(n + k))$

there is a slightly more efficient algorithm for finding a maximal matching in an unweighted bipartite graph, without using a flow network. it's slightly more complicated than Ford-Fulkerson method, so we decided to not present it.
that algorithm is called [Hopcroft–Karp algorithm](https://en.wikipedia.org/wiki/Hopcroft%E2%80%93Karp_algorithm), and its complexity is $O(|E|\sqrt|v|)$.

In [13]:
def allocate_tasks(values_mat, ret_array=False):
    """
    param values_mat: a matrix A, s.t Aij is the value of agent i, when his task is j.
    return the best makespan match v and its value. vi is the task of the agent i in the match.
    """

    tasks_num = values_mat.shape[1]

    sorted_rewards = sorted(values_mat.reshape(-1))

    low = 0
    high = len(sorted_rewards) - 1

    match = [0]

    while high > low:
        mid = int((high + low) / 2)

        reward_mat_copy = values_mat.copy()
        reward_mat_copy[reward_mat_copy > sorted_rewards[mid]] = 0
        match = maximum_bipartite_matching(csr_matrix(reward_mat_copy), perm_type='column')
        if np.sum(match[match > 0]) < tasks_num:
            high = mid
        else:
            low = mid + 1

    weights = [values_mat[i][match[i]] for i in range(len(match)) if match[i] >= 0]
    if ret_array:
        match = [match[i[0]] for i in sorted(enumerate(weights), reverse=True, key=lambda x: x[1])]
        return tuple(np.array(sorted(weights, reverse=True))), tuple(match)
    if len(weights)==0:
        ret = 0
    else:
        ret = np.max(weights)
    return ret


maximum_bipartite_matching implements the Hopcroft–Karp algorithm

#### Running The Task Allocation
now we will run the same simulation that we ran before, but in a distributed way as we saw, by using the Manhattan distance as a single agent heuristic.
first, we will create the values matrix

In [14]:
def distributed_simulation(state, h, with_prints=False):
    taxis_num = len(state[0])
    passengers_num = len(state[2])
    values_mat = np.zeros(taxis_num * passengers_num).reshape(taxis_num, passengers_num)
    for i in range(len(state[0])):  # taxi_id
        for j in range(len(state[2])):  # passenger_id
            s = deepcopy([[state[0][i]], [np.inf], [state[2][j]], [state[3][j]], [state[4][j]]])
            path = joint_simulation(s, h, print_simulation=with_prints)
            values_mat[i][j] = len(path)
    return values_mat


In [15]:
def manhattan_heuristic_single_agent(node):
    state = node.state.key
    taxi_src = state[0][0]
    passenger_src = state[2][0]
    passenger_dst = state[3][0]
    passenger_status = state[4][0]
    return manhattan_heuristic(0, taxi_src, passenger_src, passenger_dst, passenger_status)

In [16]:
taxi_env.reset()
map_problem.set_state(initial_state)
taxi_env.render()


+-------+
| : |F: |
| : |D:D|
| :_:_: |
| |P:G|P|
+-------+
Taxi0-YELLOW: Fuel: inf, Location: (2,2), Collided: False
Taxi1-RED: Fuel: inf, Location: (2,1), Collided: False
Passenger1: Location: (3, 1), Destination: (1, 3)
Passenger2: Location: (3, 3), Destination: (1, 2)
Done: False, {'taxi_0': False, 'taxi_1': False, '__all__': False}
Passengers Status's: [2, 2]


In [17]:
value_matrix = distributed_simulation(initial_state,manhattan_heuristic_single_agent,with_prints=True)
print(value_matrix)

+-------+
| : |F: |
| : |_: |
| : : : |
| | :G| |
+-------+
Taxi0-YELLOW: Fuel: inf, Location: (1,2), Collided: False
Passenger1: Location: Arrived!, Destination: (1, 2)
Done: True, {'taxi_0': True, '__all__': True}
Passengers Status's: [1]
explored_count: 9
path length: 8
[[8. 7.]
 [7. 8.]]


now, we will apply the allocate_tasks function with that values matrix

In [18]:
allocate_tasks(value_matrix,ret_array=True)[1]

(1, 0)

we got a vector V, such that Vi is the passenger that allocated to the taxi i.

## Can we generalize our solution?

what if the taxis can interact with each other?
for example, lets say we don't allow the taxis to be in the same square at the same time.
if they will plan the path separately, they couldn't predict if they will collide somewhere along the path.
in this case, we won't get an optimal solution by using this method. so we don't have a choice, but to get back to the joint graph.
but...

##### we can use our matching algorithm to create a heuristic function for the multi-agent A*, based on a heuristic of every single agent.

## Creating a multi-agent heuristic based on a given single agent heuristics
let's say we have an admissible heuristic for each taxi and passenger pair.
if we will create a matrix of values by using this heuristic values, instead of the distances values as we did before, we will get an admissible heuristic for the multi-agent system.
proof: let $G$ be a bipartite graph with the real distances, and $G'$ the bipartite graph of the heuristic distances. for every matching $M(G)$, a maximal edge of $M(G)$ is bigger than a maximal edge of $M(G')$, because all the edges in $G'$, have lower value than the corresponding edges in $G$ (the heuristic is admissible). let $M'(G)$ be the best maximal matching of the real graph, and $M''(G')$ the best maximal matching of the heuristic graph. then, $M'(G) \geq M'(G') \geq M''(G')$, where the inequality refers to the maximal edge in the matching. the first inequality is derived from the previous claim, and the second inequality is derived from the definition of the best maximal matching.

additionally, there is no other admissible heuristic for the multi-agent system that derived ONLY from these single agent's heuristics, which dominates the heuristic we defined.
proof: lets say the given single agent's heuristic is the real distances between each pair of taxi and passenger.
in that case, the defined multi-agent's heuristic is equals to the optimal solution of the makespan problem as we saw earlier. so every multi-agent's heuristic that dominates the defined one, is greater than the optimal solution. that is, its not admissible.


now we will use this matching to create a heuristics for the multi-agent A*

In [19]:
mah = MultiAgentsHeuristic(single_agent_heuristic=manhattan_heuristic,aggr_func=allocate_tasks)

path = joint_simulation(initial_state, mah, print_simulation=True)
print(len(path))

+-------+
| : |F:_|
| : |_: |
| : : : |
| | :G| |
+-------+
Taxi0-YELLOW: Fuel: inf, Location: (0,3), Collided: False
Taxi1-RED: Fuel: inf, Location: (1,2), Collided: False
Passenger1: Location: Arrived!, Destination: (1, 3)
Passenger2: Location: Arrived!, Destination: (1, 2)
Done: True, {'taxi_0': True, 'taxi_1': True, '__all__': True}
Passengers Status's: [1, 1]
explored_count: 1085
path length: 12
12


## Summary
we showed 2 different ways to solve the makespan problem. one of them was more general (the centralized one), while the other was more efficient (the decentralized one).
we showed a matching algorithm that helped us to do a task allocation for the decentralized method, and helped us to create a good heuristic for the centralized method as well.

## Further readings

our tutorial focused on the makespan problem, but there are another common task allocation's goals.
one of the most common goals is to optimize the average reward of the agents. that means, in our taxis problem, to optimize the average time of taking a passenger to his destination.
for that problem, our "best matching" definition changes to the matching with the minimal average of the edges' weight.
to find the best matching according to this definition, we have to use other matching algorithm.
a well known algorithm that solves this problem is called  [Hungarian algorithm](https://en.wikipedia.org/wiki/Hungarian_algorithm)